## Get the top 5 users with the most time spent on the web.

In [ ]:
import os

# sqlalchemy Imports
from sqlalchemy import (
    create_engine, MetaData, Table, select
)
from sqlalchemy.sql import func, desc
from sqlalchemy.engine import Connection
from sqlalchemy.sql.expression import join

# Graphing Imports
import pandas as pd
import matplotlib.pyplot as plt